In [1]:
import pandas as pd
df = pd.read_csv('/content/sample_data/04_train.csv')

from sklearn.model_selection import train_test_split

train_set, test_set_from_train = train_test_split(df, test_size=0.2, random_state=42)

# split target and features
X_train = train_set.drop('Premium Amount', axis=1)
y_train = train_set['Premium Amount'].copy()

In [2]:
%pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.0 MB/s eta 0:00:00


In [3]:
%pip install lightbgm

ERROR: Could not find a version that satisfies the requirement lightbgm (from versions: none)
ERROR: No matching distribution found for lightbgm


In [4]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from catboost import CatBoostRegressor as Catboost
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

stacking_reg = StackingRegressor(
    estimators=[
        ('LGBM', LGBMRegressor(n_estimators=73, max_depth=10, random_state=42)),
        ('xgboost', xgb.XGBRegressor(n_estimators=51, max_depth=8, random_state=42)),
        ('catboost', Catboost(n_estimators=56, depth=7, learning_rate=1, random_state=42)),
    ],
    final_estimator=LinearRegression(),
    n_jobs=-1,
    cv=3
)

stacking_reg.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


StackingRegressor(cv=3,
                  estimators=[('LGBM',
                               LGBMRegressor(max_depth=10, n_estimators=73,
                                             random_state=42)),
                              ('xgboost',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,...
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=8,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=51, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=42, ...)),
                              ('catboost',
                               <catboost.core.CatBoostRegressor object at 0x7d852f67b4f0>)],
                  final_estimator=LinearRegression(), n_jobs=-1)

In [5]:
from sklearn.metrics import root_mean_squared_error
y_pred = stacking_reg.predict(X_train)
rmsle = root_mean_squared_error(y_train, y_pred)
rmsle

1.0538494824813736

In [6]:
X_test = test_set_from_train.drop('Premium Amount', axis=1)
y_test = test_set_from_train['Premium Amount'].copy()

In [7]:
y_pred = stacking_reg.predict(X_test)
rmsle = root_mean_squared_error(y_test, y_pred)
rmsle

1.061201174511386

In [8]:
# save final prediction
import numpy as np

test_set = pd.read_csv('/content/sample_data/04_test.csv')
del test_set['Premium Amount']
y_pred = stacking_reg.predict(test_set)
y_pred = np.exp(y_pred)
original_test_set = pd.read_csv('/content/sample_data/test.csv')
id = original_test_set['id']
final_prediction = pd.DataFrame({'id': id, 'Premium Amount': y_pred})
final_prediction.to_csv('stacking_reg_06.csv', index=False)